<a href="https://colab.research.google.com/github/gogooma9/SolarGen/blob/master/src/solar_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>